In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import Window
spark = SparkSession.builder\
        .master("local[4]")\
        .appName('PySpark_Tutorial')\
        .getOrCreate()



In [2]:
df = spark.read.csv('owid-covid-data.csv',sep = ',',header = True, inferSchema = True)

pyspark.sql.dataframe.DataFrame

In [3]:
df.select(['iso_code',col('location').alias('Страна'), format_number((df['total_cases']/df['population'])*100,3)\
    .alias('Процент переболевших')])\
    .filter(~col('Страна').isin('World','Europe','European Union','Asia','South America','North America','Africa'))\
    .filter( col('date') == lit('2020-03-31'))\
    .orderBy('процент переболевших', ascending=False)\
    .show(15,truncate=False)

+--------+-------------+--------------------+
|iso_code|Страна       |Процент переболевших|
+--------+-------------+--------------------+
|VAT     |Vatican      |0.742               |
|SMR     |San Marino   |0.695               |
|AND     |Andorra      |0.487               |
|LUX     |Luxembourg   |0.348               |
|ISL     |Iceland      |0.333               |
|ESP     |Spain        |0.205               |
|CHE     |Switzerland  |0.192               |
|LIE     |Liechtenstein|0.178               |
|ITA     |Italy        |0.175               |
|MCO     |Monaco       |0.133               |
|AUT     |Austria      |0.113               |
|BEL     |Belgium      |0.110               |
|DEU     |Germany      |0.086               |
|NOR     |Norway       |0.086               |
|FRA     |France       |0.077               |
+--------+-------------+--------------------+
only showing top 15 rows



In [4]:
df.select(['date',col('location').alias('Страна'),'new_cases'])\
    .filter(~col('Страна').isin('World','Europe','European Union','Asia','South America','North America','Africa'))\
    .filter( (col('date') <= lit('2021-03-31')) & (col('date') >= lit('2021-03-25')))\
    .groupBy('Страна').agg(sum('new_cases').alias('Кол-во новых случаев'),\
                           lit('c 2021-03-25 по 2021-03-31').alias('Число'))\
    .orderBy('Кол-во новых случаев', ascending=False)\
    .show(10,truncate=False)

+-------------+--------------------+--------------------------+
|Страна       |Кол-во новых случаев|Число                     |
+-------------+--------------------+--------------------------+
|Brazil       |528736.0            |c 2021-03-25 по 2021-03-31|
|United States|448300.0            |c 2021-03-25 по 2021-03-31|
|India        |434131.0            |c 2021-03-25 по 2021-03-31|
|France       |266069.0            |c 2021-03-25 по 2021-03-31|
|Turkey       |225900.0            |c 2021-03-25 по 2021-03-31|
|Poland       |201046.0            |c 2021-03-25 по 2021-03-31|
|Italy        |144037.0            |c 2021-03-25 по 2021-03-31|
|Germany      |120656.0            |c 2021-03-25 по 2021-03-31|
|Ukraine      |95016.0             |c 2021-03-25 по 2021-03-31|
|Argentina    |78944.0             |c 2021-03-25 по 2021-03-31|
+-------------+--------------------+--------------------------+
only showing top 10 rows



In [9]:
df.withColumn('Кол-во новых случаев вчера',lit(lag('new_cases',1)\
                                                .over(Window()\
                                                .partitionBy(col('location'))\
                                                .orderBy(col('date')))))\
    .select([col ('date').alias ('Число'),\
            'Кол-во новых случаев вчера',\
             col('new_cases').alias('Кол-во новых случаев сегодня')])\
    .filter( (col('date') <= lit('2021-03-31')) & (col('date') >= lit('2021-03-25')))\
    .filter((col('location') == 'Russia'))\
    .withColumn('Дельта',col('Кол-во новых случаев сегодня')-col('Кол-во новых случаев вчера'))\
    .show(truncate=False)

+----------+--------------------------+----------------------------+------+
|Число     |Кол-во новых случаев вчера|Кол-во новых случаев сегодня|Дельта|
+----------+--------------------------+----------------------------+------+
|2021-03-25|8769.0                    |9128.0                      |359.0 |
|2021-03-26|9128.0                    |9073.0                      |-55.0 |
|2021-03-27|9073.0                    |8783.0                      |-290.0|
|2021-03-28|8783.0                    |8979.0                      |196.0 |
|2021-03-29|8979.0                    |8589.0                      |-390.0|
|2021-03-30|8589.0                    |8162.0                      |-427.0|
|2021-03-31|8162.0                    |8156.0                      |-6.0  |
+----------+--------------------------+----------------------------+------+

